In [1]:
import glob
import cv2 # import OpenCV
import pandas as pd
import numpy as np
import os

In [2]:
try:
    # identifiant Windows
    import subprocess, re
    ide = re.findall(r'[A-Z0-9-]+-[A-Z0-9-]+',str(subprocess.check_output('wmic csproduct get uuid').strip()))[0]
except OSError:
    try:
        # identifiant Linux
        import pwd
        ide = pwd.getpwuid( os.getuid() ).pw_name
    except OSError:
        # identifiant Mac OS
        import uuid
        ide = hex(uuid.getnode())

local_var = pd.read_csv('path.csv',index_col=0)

if ide in local_var.index:
    username = local_var.loc[ide,'name']
    path_dataset = local_var.loc[ide,'path_dataset']
    path_analysis = local_var.loc[ide,'path_analysis']
    path_data_prep = local_var.loc[ide,'path_data_prep']
    path_models = local_var.loc[ide,'path_models']
    print(f'  Username: {username}\n'
         +f'  Dataset directory: {path_dataset}\n'
         +f'  Saved analysis: {path_analysis}\n'
         +f'  Data preprocessed directory: {path_data_prep}\n'
         +f'  Models directory: {path_models}')
else:
    print('Computer not recognized :\n'
         f'  Please enter your computer_id {ide} and the associated parameters in \path.csv')
    
## Chemin pour aller récupérer les images stocker dans le dossier 'PBC_dataset_normal_DIB'
path = path_dataset

  Username: Caroline
  Dataset directory: /home/caroline/caroline/FORMATION/Datascientest/Projet_Blood_Cell/datasets/PBC_dataset_normal_DIB/
  Saved analysis: /home/caroline/caroline/FORMATION/Datascientest/Projet_Blood_Cell/analyses/
  Data preprocessed directory: /home/caroline/caroline/FORMATION/Datascientest/Projet_Blood_Cell/data_prep/
  Models directory: nan


In [3]:
# directory='data\\PBC_dataset_normal_DIB'
files=glob.glob(path_dataset+'*/*.jpg')
files = [filename.replace('\\','/') for filename in files]
print(len(files))

17202


In [4]:
files[210]

'/home/caroline/caroline/FORMATION/Datascientest/Projet_Blood_Cell/datasets/PBC_dataset_normal_DIB/lymphocyte/LY_607133.jpg'

In [5]:
df=pd.DataFrame({'filename':files})
df['cell_type']=df.filename.apply(lambda x: x.split('/')[-2])
df['img_name']=df.filename.apply(lambda x: x.split('/')[-1])
df['img_code']=df.filename.apply(lambda x: x.split('/')[-1].split('_')[0])
df['img_type']=df.filename.apply(lambda x: x.split('/')[-1][-3:])
df['filename']=df.filename.apply(lambda x: x.replace(path_dataset,''))
df.head()

,filename,cell_type,img_name,img_code,img_type
0,lymphocyte/LY_595308.jpg,lymphocyte,LY_595308.jpg,LY,jpg
1,lymphocyte/LY_329937.jpg,lymphocyte,LY_329937.jpg,LY,jpg
2,lymphocyte/LY_500097.jpg,lymphocyte,LY_500097.jpg,LY,jpg
3,lymphocyte/LY_995949.jpg,lymphocyte,LY_995949.jpg,LY,jpg
4,lymphocyte/LY_6217.jpg,lymphocyte,LY_6217.jpg,LY,jpg


In [6]:
# get information reading the file
# !!!! N.B. Paul : J'ai supprimé le fichier neutrophil/DS_169665.jpg qui posait des problèmes - sûrement corrompu

img_dim,img_height,img_width=[],[],[]

for file in files:
    # print(file)
    img = cv2.imread(file,cv2.IMREAD_COLOR)
    try:
        height, width = img.shape[:2]
    except AttributeError:
        height, width = 0,0 # when Open CV return an error (NoneType object while reading the file)
    
    img_dim.append(str(height) + ' x ' + str(width))
    img_height.append(height)
    img_width.append(width)

In [7]:
df=pd.concat([df,pd.DataFrame({
                  'img_dim':img_dim
                 ,'img_height':img_height
                 ,'img_width':img_width
                 ,'img_pixels':[h*w for h,w in zip(img_height,img_width)]
                })],axis=1)
df.head()

,filename,cell_type,img_name,img_code,img_type,img_dim,img_height,img_width,img_pixels
0,lymphocyte/LY_595308.jpg,lymphocyte,LY_595308.jpg,LY,jpg,363 x 360,363,360,130680
1,lymphocyte/LY_329937.jpg,lymphocyte,LY_329937.jpg,LY,jpg,363 x 360,363,360,130680
2,lymphocyte/LY_500097.jpg,lymphocyte,LY_500097.jpg,LY,jpg,363 x 360,363,360,130680
3,lymphocyte/LY_995949.jpg,lymphocyte,LY_995949.jpg,LY,jpg,363 x 360,363,360,130680
4,lymphocyte/LY_6217.jpg,lymphocyte,LY_6217.jpg,LY,jpg,363 x 360,363,360,130680


In [8]:
df.info()
df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17202 entries, 0 to 17201
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   filename    17202 non-null  object
 1   cell_type   17202 non-null  object
 2   img_name    17202 non-null  object
 3   img_code    17202 non-null  object
 4   img_type    17202 non-null  object
 5   img_dim     17202 non-null  object
 6   img_height  17202 non-null  int64 
 7   img_width   17202 non-null  int64 
 8   img_pixels  17202 non-null  int64 
dtypes: int64(3), object(6)
memory usage: 1.2+ MB


filename      0
cell_type     0
img_name      0
img_code      0
img_type      0
img_dim       0
img_height    0
img_width     0
img_pixels    0
dtype: int64

In [9]:
df.cell_type.value_counts()

neutrophil      3439
eosinophil      3117
ig              2895
platelet        2348
erythroblast    1551
monocyte        1420
basophil        1218
lymphocyte      1214
Name: cell_type, dtype: int64

In [10]:
df.img_code.value_counts()

EO            3117
PLATELET      2348
SNE           1711
BNE           1678
ERB           1551
MO            1420
BA            1218
LY            1214
MY            1137
MMY           1015
PMY            592
IG             151
NEUTROPHIL      50
Name: img_code, dtype: int64

In [11]:
df.img_dim.value_counts()

363 x 360    16749
369 x 366      250
360 x 360      198
361 x 360        2
360 x 361        1
360 x 359        1
360 x 362        1
Name: img_dim, dtype: int64

In [12]:
gb=df.groupby(['cell_type','img_code']).agg({'filename':'count'}).rename(columns={'filename':'count'})
gb

count
cell_type    img_code         
basophil     BA           1218
eosinophil   EO           3117
erythroblast ERB          1551
ig           IG            151
             MMY          1015
             MY           1137
             PMY           592
lymphocyte   LY           1214
monocyte     MO           1420
neutrophil   BNE          1678
             NEUTROPHIL     50
             SNE          1711
platelet     PLATELET     2348

In [13]:
df.rename(columns={'img_code':'cell_subtype_key'}, inplace=True)
df.head()

,filename,cell_type,img_name,cell_subtype_key,img_type,img_dim,img_height,img_width,img_pixels
0,lymphocyte/LY_595308.jpg,lymphocyte,LY_595308.jpg,LY,jpg,363 x 360,363,360,130680
1,lymphocyte/LY_329937.jpg,lymphocyte,LY_329937.jpg,LY,jpg,363 x 360,363,360,130680
2,lymphocyte/LY_500097.jpg,lymphocyte,LY_500097.jpg,LY,jpg,363 x 360,363,360,130680
3,lymphocyte/LY_995949.jpg,lymphocyte,LY_995949.jpg,LY,jpg,363 x 360,363,360,130680
4,lymphocyte/LY_6217.jpg,lymphocyte,LY_6217.jpg,LY,jpg,363 x 360,363,360,130680


In [14]:
# pour sauvegarder la table data.csv
df.to_csv('img_info.csv',index=False)

In [15]:
# vérifier si les fichiers ont les bonnes extensions et ne sont pas corrompus :

import os
from PIL import Image

folder_path = path_dataset
extensions = []
for fldr in os.listdir(folder_path):
    sub_folder_path = os.path.join(folder_path, fldr)
    for filee in os.listdir(sub_folder_path):
        file_path = os.path.join(sub_folder_path, filee)
        print('** Path: {}  **'.format(file_path), end="\r", flush=True)
        im = Image.open(file_path)
        rgb_im = im.convert('RGB')
        if filee.split('.')[1] not in extensions:
            extensions.append(filee.split('.')[1])